#### **사이킷런을 통한 데이터 전처리 가이드 번역**

참고 = `https://towardsdatascience.com/preprocessing-with-sklearn-a-complete-and-comprehensive-guide-670cb98fcfb9`

본 내용은 sklearn v0.20.0을 기준으로 작성되었음 <br>
사이킷런에 제공하는 class(기능)과 도움이되는 다른 라이브러리를 포함함
다음과 같은 주제를 다룸 
1. 결측치 
2. 다항식 변환
3. 범주형 변수
4. 수치형 변수
5. Custom 기능
6. 변수 스케일링
7. 정규화 (Normalization)

####  결측치

결측치 핸들링은 전처리 작업에서 매우 중요 : 모델의 결과에 상당한 영향을 줄 수 있음

결측히 핸들링에 고려해야하는 사항
* 결측치가 있는가?
* 주어진 데이터에서 결측치는 어떻게 표현되고 있는가?
* 결측치를 그대로 유지해도 되는가?
* 결측치를 다른 값으로 대체해야 하는가?
* 어떤 값으로 결측치를 대체해야 하는가?

Chris Albon (Machine Learning with Python Cookbook)에 따르면, 결측치가 완전히 랜덤한 경우라면 해당 결측치는 어떤 정보를 내포하고 있지 않다고 보고 제거할 수 있다. 다만, 랜덤하지 않다면 해당 결측치는 어떠한 정보를 내포하고 있다고 볼 수 있으므로 다른 값으로 대체하여 표현되어야 한다고 말함

In [44]:
# Example - 결측치 확인/제거
import numpy as np
import pandas as pd
X = pd.DataFrame(
    np.array([5,7,8, np.NaN, np.NaN, np.NaN, -5,
              0,25,999,1,-1, np.NaN, 0, np.NaN])\
              .reshape((5,3)))
X.columns = ['f1', 'f2', 'f3'] #feature 1, feature 2, feature 3
X

,f1,f2,f3
0,5.0,7.0,8.0
1,NaN,NaN,NaN
2,-5.0,0.0,25.0
3,999.0,1.0,-1.0
4,NaN,0.0,NaN


In [45]:
# Example - 결측치 확인/제거
X.dropna(axis=0, thresh=1, inplace=True)
X.reset_index(inplace=True)
X.drop(['index'], axis=1, inplace=True)






X

,f1,f2,f3
0,5.0,7.0,8.0
1,-5.0,0.0,25.0
2,999.0,1.0,-1.0
3,NaN,0.0,NaN


In [49]:
from sklearn.impute import MissingIndicator
X.replace({999.0 : np.NaN}, inplace=True)
indicator = MissingIndicator(missing_values=np.NaN)
indicator = indicator.fit_transform(X)
indicator = pd.DataFrame(indicator, columns=['m1', 'm3'])
indicator

,m1,m3
0,False,False
1,False,False
2,True,False
3,True,True


In [51]:
# Example - 결측치 대체 - SimpleImputer 사용
# `mean` `most_frequent` `median` `constant`
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit_transform(X)  # 결과: numpy array 

array([[ 5.        ,  7.        ,  8.        ],
       [-5.        ,  0.        , 25.        ],
       [ 0.        ,  1.        , -1.        ],
       [ 0.        ,  0.        , 10.66666667]])

In [53]:
# Example - 결측치 대체 - 판다스에서 직접 대체하는 방법
X.fillna(X.mean(), inplace=True)
X

,f1,f2,f3
0,5.0,7.0,8.000000
1,-5.0,0.0,25.000000
2,0.0,1.0,-1.000000
3,0.0,0.0,10.666667


#### 다항식 변환

In [57]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, interaction_only=True)
polynomials = pd.DataFrame(poly\
                           .fit_transform(X), 
                           columns=['0','1','2','3', 
                                    'p1', 'p2', 'p3', 'p4'])\
                                        [['p1', 'p2', 'p3', 'p4']]

In [58]:
polynomials

,p1,p2,p3,p4
0,35.0,40.0,56.0,280.0
1,-0.0,-125.0,0.0,-0.0
2,0.0,-0.0,-1.0,-0.0
3,0.0,0.0,0.0,0.0


In [59]:
X = pd.concat([X, indicator, polynomials], axis=1)
X

,f1,f2,f3,m1,m3,p1,p2,p3,p4
0,5.0,7.0,8.000000,False,False,35.0,40.0,56.0,280.0
1,-5.0,0.0,25.000000,False,False,-0.0,-125.0,0.0,-0.0
2,0.0,1.0,-1.000000,True,False,0.0,-0.0,-1.0,-0.0
3,0.0,0.0,10.666667,True,True,0.0,0.0,0.0,0.0


####  범주형 변수

범주형 변수는 머신러닝에 모델의 입력데이터로 사용될 수 없으므로 수치형으로 표현되어야 한다 <br>

범주형 변수 작업에서 고려해야하는 사항
* 변수가 ordinal인가? - `OrdinalEncoder`, `LabelEncoder`
* 변수가 nominal인가? - `OneHotEncoder`

In [229]:
# Example - 범주형 변수 작업
X = pd.DataFrame(np.array([
    'M', 'O-', 'medium',
    'M', 'O-', 'high',
    'F', 'O+', 'high',
    'F', 'AB', 'low',
    'F', 'B+', 'low']
).reshape((5,3)))
X.columns = ['sex', 
             'blood_type', 
             'edu_level']

In [230]:

X

,sex,blood_type,edu_level
0,M,O-,medium
1,M,O-,high
2,F,O+,high
3,F,AB,low
4,F,B+,low


In [165]:
# Example - 범주형 변수 작업 - OrdinalEncoder 사용하는 방법
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
X.edu_level = encoder.fit_transform(X.edu_level.values.reshape(-1, 1))
print('인코딩된 카테고리', encoder.categories_)
#encoder.inverse_transform(X.edu_level.values.reshape(-1, 1))
X

인코딩된 카테고리 [array(['high', 'low', 'medium'], dtype=object)]


,sex,blood_type,edu_level
0,M,O-,2.0
1,M,O-,0.0
2,F,O+,0.0
3,F,AB,1.0
4,F,B+,1.0


In [231]:
# Example - 범주형 변수 작업 - LabelEncoder 사용하는 방법
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder()
X['edu_encode'] = encoder.fit_transform(X.edu_level)

print('인코딩된 카테고리', encoder.classes_)
X


인코딩된 카테고리 ['high' 'low' 'medium']


,sex,blood_type,edu_level,edu_encode
0,M,O-,medium,2
1,M,O-,high,0
2,F,O+,high,0
3,F,AB,low,1
4,F,B+,low,1


In [160]:
# Example - 범주형 변수 작업
X = pd.DataFrame(np.array([
    'M', 'O-', 'medium',
    'M', 'O-', 'high',
    'F', 'O+', 'high',
    'F', 'AB', 'low',
    'F', 'B+', np.NaN]
).reshape((5,3)))
X.columns = ['sex', 
             'blood_type', 
             'edu_level']

In [152]:
# Example - 범주형 변수 작업 - 판다스를 사용하는 방법
cat = pd.Categorical(X.edu_level, 
                     categories=['missing', 'low', 
                                 'medium', 'high'], 
                     ordered=True)
cat.fillna('missing')
labels, unique = pd.factorize(cat, sort=True)
X.edu_level = labels
X

,sex,blood_type,edu_level
0,M,O-,1
1,M,O-,2
2,F,O+,2
3,F,AB,0
4,F,B+,-1


In [169]:
# Example - 범주형 변수 작업 - OneHotEncoder 사용하는 방법
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(dtype=np.int, sparse=True)
nominals = pd.DataFrame(
    onehot.fit_transform(X[['sex', 'blood_type']]).toarray(),
    columns=['F', 'M', 'AB', 'B+','O+', 'O-'])
nominals['edu_level'] = X.edu_level
nominals

,F,M,AB,B+,O+,O-,edu_level
0,0,1,0,0,0,1,2
1,0,1,0,0,0,1,0
2,1,0,0,0,1,0,0
3,1,0,1,0,0,0,1
4,1,0,0,1,0,0,1


In [246]:
# Example - 범주형 변수 작업 - OneHotEncoder 사용하는 방법
onehot = OneHotEncoder(sparse=False)
onehot.fit_transform(X.edu_level.values.reshape(-1,1))
print(onehot.categories_)

[array(['high', 'low', 'medium'], dtype=object)]


In [260]:
onehot.categories_[0]

array(['high', 'low', 'medium'], dtype=object)

In [261]:
onehot_edu = onehot.fit_transform(X.edu_level.values.reshape(-1,1))
onehot_edu = pd.DataFrame(onehot_edu, columns=onehot.categories_[0])
X_ = pd.concat([X, onehot_edu], axis=1)
X_

,sex,blood_type,edu_level,edu_encode,sex_enc,sex_encode,high,low,medium
0,M,O-,medium,2,1,1,0.0,0.0,1.0
1,M,O-,high,0,1,1,1.0,0.0,0.0
2,F,O+,high,0,0,0,1.0,0.0,0.0
3,F,AB,low,1,0,0,0.0,1.0,0.0
4,F,B+,low,1,0,0,0.0,1.0,0.0


In [233]:
# Example - 범주형 변수 작업 - LabelEncoder 사용하는 방법
le = LabelEncoder()
le.fit_transform(X.sex)
print(le.classes_)
#le.inverse_transform([1,1,0])
X['sex_encode'] = le.fit_transform(X.sex)
X

['F' 'M']


,sex,blood_type,edu_level,edu_encode,sex_enc,sex_encode
0,M,O-,medium,2,1,1
1,M,O-,high,0,1,1
2,F,O+,high,0,0,0
3,F,AB,low,1,0,0
4,F,B+,low,1,0,0


####  수치형 변수

이산화 (Binarization/Discretization)

####  Custom 기능

#### 변수 스케일링

변수 스케일링을 하기 전에 반드시 train/test 데이터를 나눠야 함
train/test 나누기 이전에 변수 스케일링을 하게 되면, 

Standarization
대부분의 학습 알고리즘의 목점 함수의 가정: 변수의 평균이 0(또는 0 주변)
1. StandardScaler
2. MinMaxScaler
3. MaxAbsScaler
4. RobustScaler


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(X.f3.values.reshape(-1, 1))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-3,3))
scaler.fit_transform(X.f3.values.reshape(-1, 1))

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
scaler.fit_transform(X.f3.values.reshape(-1, 1))

In [ ]:
from sklearn.preprocessing import RobustScaler
robust = RobustScaler(quantile_range = (0.1,0.9))
robust.fit_transform(X.f3.values.reshape(-1, 1))

#### 정규화